* [Yelp, Not Kaggle Data Source](https://www.yelp.com/dataset/download)
* [Data Dictionary](https://www.yelp.com/dataset/documentation/main)

* [ ] Get data into csv to work in pandas
* [ ] Perform sentiment classification with Hugging Face model
* [ ] Extract from text key words that resulted in sentiment class
* [ ] topic modeling?
* [ ] other advanced analytics for reviews using LLM/HuggingFace
* [ ] compare competition side-by-side (Japanese restaurants)
* [ ] get criteria for ranking/qualities in school based on aggregation of reviews

In [1]:
!pip install duckdb jupysql --quiet

In [2]:
import pandas as pd
import duckdb

%load_ext sql
conn = duckdb.connect()
%sql conn --alias duckdb

Deploy Flask apps for free on Ploomber Cloud! Learn more: https://ploomber.io/s/signup


# Get relevant data (business, review, user)
Data Ingestion credit: https://towardsdatascience.com/converting-yelp-dataset-to-csv-using-pandas-2a4c8f03bd88

In [3]:
business_json_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_business.json'
df_b = pd.read_json(business_json_path, lines=True)
df_b.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


# Clean biz file

In [4]:
# 1 = open, 0 = closed
df_b = df_b[df_b['is_open']==1] # getting biz in operation only


drop_columns = ['hours','is_open']
df_b.drop(drop_columns, axis=1, inplace=True)
df_b.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma..."


In [5]:
# Getting restaurant related businesses
# get task and prompt columns only
query = """
select *
from df_b
where lower(categories) like '%restaurants%'
"""
biz_rest =duckdb.query(query).df()
biz_rest.head()

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeatin...","Restaurants, Food, Bubble Tea, Coffee & Tea, B..."
1,CF33F8-E6oudUQ46HnavjQ,Sonic Drive-In,615 S Main St,Ashland City,TN,37015,36.269593,-87.058943,2.0,6,"{'key': ['BusinessParking', 'BusinessAcceptsCr...","Burgers, Fast Food, Sandwiches, Food, Ice Crea..."
2,bBDDEgkFA1Otx9Lfe7BZUQ,Sonic Drive-In,2312 Dickerson Pike,Nashville,TN,37207,36.208102,-86.768170,1.5,10,"{'key': ['RestaurantsAttire', 'RestaurantsGood...","Ice Cream & Frozen Yogurt, Fast Food, Burgers,..."
3,eEOYSgkmpB90uNA7lDOMRA,Vietnamese Food Truck,,Tampa Bay,FL,33602,27.955269,-82.456320,4.0,10,"{'key': ['Alcohol', 'OutdoorSeating', 'Restaur...","Vietnamese, Food, Restaurants, Food Trucks"
4,il_Ro8jwPlHresjw9EGmBg,Denny's,8901 US 31 S,Indianapolis,IN,46227,39.637133,-86.127217,2.5,28,"{'key': ['RestaurantsReservations', 'Restauran...","American (Traditional), Restaurants, Diners, B..."


In [6]:
biz_rest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34987 entries, 0 to 34986
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   business_id   34987 non-null  object 
 1   name          34987 non-null  object 
 2   address       34987 non-null  object 
 3   city          34987 non-null  object 
 4   state         34987 non-null  object 
 5   postal_code   34987 non-null  object 
 6   latitude      34987 non-null  float64
 7   longitude     34987 non-null  float64
 8   stars         34987 non-null  float64
 9   review_count  34987 non-null  int64  
 10  attributes    34547 non-null  object 
 11  categories    34987 non-null  object 
dtypes: float64(3), int64(1), object(8)
memory usage: 3.2+ MB


# Bring in review data and retain businesses with reviews only

In [7]:
review_json_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_review.json'

size = 1000000
review = pd.read_json(review_json_path, lines=True,
                      dtype={'review_id':str,'user_id':str,
                             'business_id':str,'stars':int,
                             'date':str,'text':str,'useful':int,
                             'funny':int,'cool':int},
                      chunksize=size)

In [8]:
# There are multiple chunks to be read
chunk_list = []
for chunk_review in review:
    # Drop columns that aren't needed
    chunk_review = chunk_review.drop(['useful','funny','cool'], axis=1)
    # Renaming column name to avoid conflict with business overall star rating
    chunk_review = chunk_review.rename(columns={'stars': 'review_stars'})
    # Inner join with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(biz_rest, chunk_review, on='business_id', how='inner')
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

540141 out of 1,000,000 related reviews
549947 out of 1,000,000 related reviews
524365 out of 1,000,000 related reviews
529735 out of 1,000,000 related reviews
546958 out of 1,000,000 related reviews
544052 out of 1,000,000 related reviews
538572 out of 1,000,000 related reviews


In [9]:
df.head()

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,attributes,categories,review_id,user_id,review_stars,text,date
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeatin...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",BXQcBN0iAi1lAUxibGLFzA,6_SpY41LIHZuIaiDs5FMKA,4,This is nice little Chinese bakery in the hear...,2014-05-26 01:09:53
1,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeatin...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",uduvUCvi9w3T2bSGivCfXg,tCXElwhzekJEH6QJe3xs7Q,4,This is the bakery I usually go to in Chinatow...,2013-10-05 15:19:06
2,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeatin...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",a0vwPOqDXXZuJkbBW2356g,WqfKtI-aGMmvbA9pPUxNQQ,5,"A delightful find in Chinatown! Very clean, an...",2013-10-25 01:34:57
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeatin...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",MKNp_CdR2k2202-c8GN5Dw,3-1va0IQfK-9tUMzfHWfTA,5,I ordered a graduation cake for my niece and i...,2018-05-20 17:58:57
4,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,"{'key': ['RestaurantsDelivery', 'OutdoorSeatin...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...",D1GisLDPe84Rrk_R4X2brQ,EouCKoDfzaVG0klEgdDvCQ,4,HK-STYLE MILK TEA: FOUR STARS\n\nNot quite su...,2013-10-25 02:31:35


In [10]:
len(biz_rest), len(df) # length of df should be longer with reviews fanning out on businesses (multiple reviews)

(34987, 3773770)

In [11]:
# which restaurants have the most reviews
query = """
select name
, count(*) as total_reviews
from df
group by 1
order by 2 desc
limit 10
"""
duckdb.query(query).df()

,name,total_reviews
0,McDonald's,17309
1,Chipotle Mexican Grill,9438
2,First Watch,9076
3,Acme Oyster House,8461
4,Taco Bell,8320
5,Chick-fil-A,8315
6,Oceana Grill,7516
7,Buffalo Wild Wings,7063
8,Panera Bread,6906
9,Domino's Pizza,6335


# Get user data
Might be useful to pivot by user to see how they perform reviews in future iteration

## Need to perform chunking with user data as well

In [12]:
user_json_path = '/kaggle/input/yelp-dataset/yelp_academic_dataset_user.json'

size = 900000
user = pd.read_json(user_json_path, lines=True,
                      dtype={'user_id':str,'name':str,
                             'review_count':int,'yelping_since':str,
                             'friends':object,'useful':int,
                             'funny':int,'cool':int,
                             'fans':int, 'elite':object,
                            'average_stars':float, 'compliment_hot':int,
                            'compliment_more':int, 'compliment_profile':int,
                            'compliment_cute':int, 'compliment_list':int,
                            'compliment_note':int, 'compliment_plain':int,
                            'compliment_cool':int, 'compliment_funny':int,
                            'compliment_writer':int, 'compliment_photos':int},
                      chunksize=size)

In [13]:
# There are multiple chunks to be read
chunk_list = []
for chunk_user in user:
    # Renaming column name to avoid conflict with business name
    chunk_user = chunk_user.rename(columns={'name': 'customer_name', 'review_count': 'customer_review_count'})
    # Inner join with edited business file so only reviews related to the business remain
    chunk_merged = pd.merge(df, chunk_user, on='user_id', how='inner') # only keep user that have made restaurant review
    # Show feedback on progress
    print(f"{chunk_merged.shape[0]} out of {size:,} related reviews")
    chunk_list.append(chunk_merged)
# After trimming down the review file, concatenate all relevant data back to one dataframe
df = pd.concat(chunk_list, ignore_index=True, join='outer', axis=0)

2984785 out of 900,000 related reviews
690844 out of 900,000 related reviews
98134 out of 900,000 related reviews


In [14]:
df.head(1)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,...,compliment_more,compliment_profile,compliment_cute,compliment_list,compliment_note,compliment_plain,compliment_cool,compliment_funny,compliment_writer,compliment_photos
0,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,...,0,0,0,0,0,0,0,0,1,0


In [15]:
list(df)

['business_id',
 'name',
 'address',
 'city',
 'state',
 'postal_code',
 'latitude',
 'longitude',
 'stars',
 'review_count',
 'attributes',
 'categories',
 'review_id',
 'user_id',
 'review_stars',
 'text',
 'date',
 'customer_name',
 'customer_review_count',
 'yelping_since',
 'useful',
 'funny',
 'cool',
 'elite',
 'friends',
 'fans',
 'average_stars',
 'compliment_hot',
 'compliment_more',
 'compliment_profile',
 'compliment_cute',
 'compliment_list',
 'compliment_note',
 'compliment_plain',
 'compliment_cool',
 'compliment_funny',
 'compliment_writer',
 'compliment_photos']

In [16]:
df.to_csv('yelp_final.csv')